מגישות:
נועה פרומקיס,
נועה זילברשטיין,
חן אופיר גולן
ושיר טבת 

הנתונים לקחו מאתר קאגל:https://www.kaggle.com/andrewmvd/data-engineer-jobs

הורדה וייבוא של כל הספריות שבהם נשתמש במהלך הקובץ

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import PercentFormatter
import seaborn as sns
from os import path
from PIL import Image


In [ ]:
data = 'https://raw.githubusercontent.com/shirtevet/project_Data-Engineer-Jobs/main/DataEngineer.csv'
Data_Engineer= pd.read_csv(data)

Data_Engineer

הצגת הנתונים בטבלה

In [ ]:
Data_Engineer.shape

In [ ]:
Data_Engineer["Job Title"].value_counts()

In [ ]:
Data_Engineer.columns

בקובץ מופיע הערך חסר כ"1-" ולכן שינינו את כל הווריציות של ערכים אלה לנאן    

In [ ]:
Data_Engineer.replace(['-1'], [np.nan], inplace=True)
Data_Engineer.replace(['-1.0'], [np.nan], inplace=True)
Data_Engineer.replace([-1], [np.nan], inplace=True)
Data_Engineer

בדיקה שאכן מילאנו את הערכים החסרים 

In [ ]:
Data_Engineer.isnull().sum()

טיפול בערכים חסרים

טיפול בערכים חסרים:
את הערכים החסרים בעמודת הדירוג- מילאנו בממוצע הדירוגים (כך שזה לא ישפיע על הממוצע הכללי)

את הערכים החסרים בעמודת מיקום ההנהלה של החברה- מילאנו במיקום החברה (מתוך היגיון)



In [ ]:
Data_Engineer["Rating"].fillna(Data_Engineer["Rating"].mean() , inplace=True)
Data_Engineer["Headquarters"].fillna(Data_Engineer["Location"] , inplace=True)
Data_Engineer

טיפול במשכורת:
הורדת המלל המיותר בכדי שנוכל להשתמש במספרי המשכורות בלבד

In [ ]:

Data_Engineer['Salary Estimate'] = Data_Engineer['Salary Estimate'].str.replace('(','').str.replace(')','').str.replace('Glassdoor est.','').str.replace('Employer est.','')


הוספת 2 עמודות חדשות, במקום עמודת הטווח של המשכורת. עמודה לכל קצה טווח- אחד למינימום ואחד למקסימום (כאשר מה שמפריד בין הערכים זה "-")
בנוסף, הורדת רווחים וסימנים כדי שנוכל להשתמש במספרים לחישובים.

In [ ]:
Data_Engineer['Minimom Salary'],Data_Engineer['Maximom Salary'] = Data_Engineer['Salary Estimate'].str.split('-').str
Data_Engineer['Minimom Salary'] = Data_Engineer['Minimom Salary'].str.strip(' ').str.strip('$').str.strip('K').fillna(0).astype(int)
Data_Engineer['Maximom Salary'] = Data_Engineer['Maximom Salary'].str.strip(' ').str.strip('$').str.strip('K').fillna(0).astype(int)
Data_Engineer

<font size="5"> תובנה ראשונה  </font>


מיינו את כל העבודות על פי הכנסת החברה.
הדפסנו היסטוגרמה וגרף נקודות של רווח החברה ולמולו המשכורת של העובד
התובנה היא כי ניתן לראות שלא בהכרח ישנה משכורת גבוהה יותר בחברה רווחית יותר

In [ ]:
count_rev = Data_Engineer.groupby('Revenue')[['Job Title']].count().reset_index().rename(columns={'Job Title':'Count'}).sort_values('Count', ascending=False).reset_index(drop=True)
count_rev = count_rev.merge(Data_Engineer, on='Revenue',how='left')
#sns.set(style="whitegrid")
f, (ax_bar, ax_point) = plt.subplots(ncols=2, sharey=True, gridspec_kw= {"width_ratios":(0.8,1)},figsize=(16,6))
sns.barplot(x='Count',y='Revenue',data=count_rev ,ax=ax_bar)
sns.pointplot(x='Minimom Salary',y='Revenue',data=count_rev , join=False,ax=ax_point).set(ylabel="",xlabel="Salary in k($)")


<font size="5"> תובנה שנייה  </font>

המשך סידור העמודות והצגת פרטים

הוספת עמודה של משכורת ממוצעת (לפי מינימום ומקסימום חלקי שתיים)

In [ ]:
Data_Engineer['Mean_Salary'] = (Data_Engineer['Minimom Salary']+Data_Engineer['Maximom Salary'])/2 
Data_Engineer

In [ ]:
box_rating = sns.boxplot(x=Data_Engineer["Rating"])
box_rating.set_title("boxplot of company rating")

In [ ]:
Data_Engineer.Rating.plot(kind='hist')
plt.title('Rating')
plt.grid
plt.show()

הגרפים המוצגים לעיל מראים את התפלגות הדירוג לפי טבלת הנתונים    
לפי גרפים אלו, ניתן לראות כי מירב החברות קיבלו דירוג הנע בין 3.5-4 מתוך טווח של 1-5

In [ ]:
Data_Engineer.Mean_Salary.plot(kind='hist')
plt.title('Mean Salary Estimate')
plt.xlabel('salary')
plt.grid
plt.show()

In [ ]:
box_salary = sns.boxplot(x=Data_Engineer["Mean_Salary"])
box_salary.set_title("boxplot of mean salary")

הגרפים המוצגים לעיל מראים את התפלגות המשכורת לפי טבלת הנתונים   
על בסיס גרפים אלו ניתן לראות כי השכר הממוצע לעובד נע בין 75-115 דולר

In [ ]:
Data_Engineer[['Rating','Mean_Salary']]

עמודת דירוג החברה בהשוואה לעמודת השכר הממוצע לעובד

In [ ]:
fig = plt.figure(figsize=(15,5))
sns.barplot(x='Rating', y='Mean_Salary', data=Data_Engineer)
plt.ylabel('Salary (USD)')
plt.xlabel('Company Rating ( According to Glassdoor website)')
plt.title('Company Ratings vs Mean Salary',fontsize=18)
plt.show()

הסבר התובנה - כאשר חשבנו על הדירוג אותו קיבלה החברה והשכר של העובד בה, מה שמתבקש הוא להסיק כי ישנו קשר ישיר בין המשתנים. ניתן לחשוב כי השכר של העובד יעלה ככל שהחברה תצליח יותר, כלומר תקבל דירוג גבוה יותר, זאת על בסיס הנטייה לחשוב כי דירוג גבוה מוביל להצלחת החברה, וכי הצלחת החברה מובילה להכנסה גבוהה. הכנסה זו תתחלק בין העובדים ומכאן כי ככל שהדירוג גבוה, הכנסת העובד גבוהה
אמנם, על בסיס ניתוח הנתונים שביצענו במסגרת מחקר על הדאטה בתחומים אלה, ניתן לראות בבירור כי הסקתנו הייתה שגויה.לא קיים קשר בין גובה הדירוג שהחברה קיבלה לבין השכר הממוצע של העובד. לפי הדיאגרמה המראה את הקשר בין הדירוג לשכר, ניכר כי רמת השכר מתפלגת בצורה כמעט אחידה בהשוואה לדירוג החברה. ניתן להבין שבחברה בעלת דירוג נמוך ובחברה בעלת דירוג גבוה, שכר העובד יכול להיות זהה 
בנוסף, ניתן לאשש תובנה זו באמצעות העובד שקיבל בממוצע את השכר הגבוה ביותר, עובד זה נמצא בחברה שקיבלה ציון נמוך מאוד בדירוג, העומד על 1.5 מתוך 5

<font size="5"> תובנה שלישית  </font>

הגרף המוצג מראה את התפלגות התפקידים הנפוצים בשוק העבודה לפי קובץ הנתונים

In [ ]:
Data_Engineer_sectors = Data_Engineer.Sector.value_counts().sort_values(ascending=False).head(n=10)

axs = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))
Data_Engineer_sectors.plot.bar(x='sectors and industries', y='%', rot=0)

לפי גרף זה ניתן לראות כי התחום בו הביקוש הגבוה ביותר לעובדים הינו תחום טכנולוגיית המידע. לאחריו, מגיע תחום השירותים הפיננסיים

In [ ]:
fig = plt.figure(figsize=(15,5))
sns.barplot(x='Mean_Salary', y='Sector', data=Data_Engineer)
plt.ylabel('Sector ( According to Glassdoor website)')
plt.xlabel('Salary (USD)')
plt.title('Sector vs Mean Salary',fontsize=20)
plt.show()

בגרפים המוצגים לעיל ניתן לראות כי אין קשר בין כמות האנשים המועסקים בתחום לבין המשכורת שהם מרוויחים.
לדוגמה, בתחום המדיה מרוויחים העובדים משכורת ממוצעת בין הגבוהות בענף. לעומת זאת, בתחום זה אין היצע גדול של עובדים.
דוגמה נוספת לכך היא שבתחום טכנולוגיית המידע ובתחום השירותים הפיננסיים, כמות העובדים היא הגדולה ביותר ואילו המשכורת בתחומים אלו הינה ממוצעת 